In [ ]:
#Setting tensorflow to use GPU to train
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
#importing required components
import numpy as np
import keras.backend as K
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Layer, Flatten, Input, BatchNormalization, Activation, Dropout
from keras_adabound import AdaBound

In [ ]:
#Using Image Data Generator to augment and feed images to the model
datagen=ImageDataGenerator(rescale=1./255., validation_split=0.1, shear_range=0.1, zoom_range = 0.2, width_shift_range=10, height_shift_range=15, horizontal_flip = True, fill_mode = "nearest")

train_gen = datagen.flow_from_directory(directory = r"D:/Crop Field Dataset/train", subset = "training", batch_size = 32, seed = 42, shuffle = True, class_mode = "categorical", target_size=(227,227))

valid_gen = datagen.flow_from_directory(directory = r"D:/Crop Field Dataset/train", subset = "validation", batch_size = 32, seed = 42, shuffle = True, class_mode = "categorical", target_size=(227,227))

In [ ]:
#Calculating number of steps in each epoch
TRAIN_STEP = train_gen.n//train_gen.batch_size
VALID_STEP = valid_gen.n//valid_gen.batch_size

In [ ]:
#Constructing the neural network model
AlexNet = Sequential()

#1st Convolutional Layer
AlexNet.add(Conv2D(filters=96, input_shape=(227,227,3), kernel_size=(11,11), strides=(4,4), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#4th Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#5th Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
AlexNet.add(Flatten())
# 1st Fully Connected Layer
AlexNet.add(Dense(4096, input_shape=(32,32,3,), kernel_regularizer=tf.keras.regularizers.l2(0.001)))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
# Add Dropout to prevent overfitting
AlexNet.add(Dropout(0.5))

#2nd Fully Connected Layer
AlexNet.add(Dense(4096))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.5))

AlexNet.add(Dense(2048))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.5))

AlexNet.add(Dense(1024))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.5))

#3rd Fully Connected Layer
AlexNet.add(Dense(1000, kernel_regularizer=tf.keras.regularizers.l2(0.001)))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.5))

#Output Layer
AlexNet.add(Dense(9))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('softmax'))

In [ ]:
#Printing the layers in the model
AlexNet.summary()

In [ ]:
#Setting up required loss function and optimizers
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)

Adam_opt = tf.keras.optimizers.Adam(learning_rate = lr_schedule)
Adab=AdaBound(lr=1e-3, final_lr=0.01)
loss_fun = tf.keras.losses.CategoricalCrossentropy()

In [ ]:
#Creating a Callback to suspend training when required accuracy is reached
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.8040):
            print("\nReached 75.0% accuracy so cancelling training!")
            self.model.stop_training = True
callbacks = myCallback()

In [ ]:
#Training for 50 epochs with Adam Optimizer
AlexNet.compile(optimizer = Adam_opt, loss = loss_fun, metrics = ['accuracy'])
history = AlexNet.fit(train_gen, validation_data = valid_gen, epochs = 50, steps_per_epoch = TRAIN_STEP, validation_steps = VALID_STEP, callbacks = [callbacks])

In [ ]:
#Training for another 50 epochs with Adabound Optimizer
AlexNet.compile(optimizer = Adab, loss = loss_fun, metrics = ['accuracy'])
history2 = AlexNet.fit(train_gen, validation_data = valid_gen, epochs = 50, steps_per_epoch = TRAIN_STEP, validation_steps = VALID_STEP, callbacks = [callbacks])

In [ ]:
#Saving the model
AlexNet.save(r"D:/model_80.h5")

In [ ]:
#Printing the labels of classes
label_map = (train_gen.class_indices)
print(label_map)

In [ ]:
#Converting model to .tflite and quantizing it to use in mobile app
new_model= tf.keras.models.load_model(filepath=r"D:/model_80.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(new_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()
with open(r"D:/quantized_model.tflite", "wb") as f:
    f.write(quantized_and_pruned_tflite_model)

In [ ]:
#Testing the model
import cv2

# read and resize the image
img = cv2.imread(r"D:\Crop Field Dataset\test\corn\test_img.jpg")
new_img = cv2.resize(img, (227, 227)).astype('float32')
new_img = new_img/255.
new_img = new_img.reshape(-1, 227, 227, 3)

#Predict and show result
arr = new_model.predict(new_img)
print(np.argmax(arr))